In [1]:
import tensorflow as tf
from tensorflow import keras

import numpy as np

from rb_equivariant_cnn import RB3D_Conv
from rb_equivariant_gcnn import RB3D_G_Conv

2024-07-31 23:18:34.603594: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Setup

In [2]:
RB_CHANNELS = 4
HORIZONTAL_SIZE = 64
HEIGHT = 32

BATCH_SIZE = 64

# 3D Convolution
- Equivariant to horizontal translations
- No vertical parameter sharing
- Supports horizontal wrap (reflect) padding
    - Makes sense when using peridoc boundary conditions for Rayleigh-Bénard
    - Attention: This (as well as SAME padding) may destroy exact rotation equivariance (nevertheless preferable in praxis)
- Supports stride (including vertical stride)
- Uses 2D convolutions under the hood

In [33]:
model = keras.Sequential([
            keras.layers.InputLayer(shape=(HORIZONTAL_SIZE, HORIZONTAL_SIZE, HEIGHT, RB_CHANNELS),
                                    batch_size=BATCH_SIZE),
            
            RB3D_Conv(v_ksize=5, h_ksize=3, channels=RB_CHANNELS, padding='REFLECT', strides=(2,2,2), name='Conv1'),
            RB3D_Conv(v_ksize=5, h_ksize=3, channels=RB_CHANNELS, padding='REFLECT', strides=(2,2,1), name='Conv2'),
            RB3D_Conv(v_ksize=5, h_ksize=3, channels=RB_CHANNELS, padding='REFLECT', strides=(2,2,1), name='Conv3'),
            RB3D_Conv(v_ksize=5, h_ksize=3, channels=RB_CHANNELS, padding='REFLECT', strides=(2,2,1), name='Conv4'),
        ])

# output shape: batch_size, width, depth, height, channels
model.summary()

Model: "sequential_27"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Conv1 (RB3D_Conv)               │ (64, 33, 33, 14, 4)    │        10,136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Conv2 (RB3D_Conv)               │ (64, 17, 17, 10, 4)    │         7,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Conv3 (RB3D_Conv)               │ (64, 9, 9, 6, 4)       │         4,344 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Conv4 (RB3D_Conv)               │ (64, 5, 5, 2, 4)       │         1,448 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,168 (90.50 KB)

 Trainable params: 23,168 (90.50 KB)

 Non-trainable params: 0 (0.00 B)

# 3D D4 Group Equivariant Convolution
- Equivariant to all symmetries of 3D Rayleigh-Bénard:
    - rotations around a vertical axis
    - reflections through a vertical plane
    - horizontal translations
- No vertical parameter sharing
- Supports horizontal wrap (reflect) padding
    - Makes sense when using peridoc boundary conditions for Rayleigh-Bénard
    - Attention: This (as well as SAME padding) may destroy exact rotation equivariance (nevertheless preferable in praxis)
- Supports stride (including vertical stride)
- Uses 2D convolutions under the hood

In [32]:
G = 'D4' # 'C4' for rotations or 'D4' for rotations and reflections
model = keras.Sequential([
            keras.layers.InputLayer(shape=(HORIZONTAL_SIZE, HORIZONTAL_SIZE, HEIGHT, RB_CHANNELS),
                                    batch_size=BATCH_SIZE),
            # add transformation dimension
            keras.layers.Reshape((HORIZONTAL_SIZE, HORIZONTAL_SIZE, 1, HEIGHT, RB_CHANNELS)), 
            
            RB3D_G_Conv('Z2', G, v_ksize=5, h_ksize=3, channels=RB_CHANNELS, padding='REFLECT', strides=(2,2,2), 
                        name=f'Lift_{G}_Conv1'),
            RB3D_G_Conv(G, G, v_ksize=5, h_ksize=3, channels=RB_CHANNELS, padding='REFLECT', strides=(2,2,1),
             name=f'{G}_Conv2'),
            RB3D_G_Conv(G, G, v_ksize=5, h_ksize=3, channels=RB_CHANNELS, padding='REFLECT', strides=(2,2,1),
             name=f'{G}_Conv3'),
            RB3D_G_Conv(G, G, v_ksize=5, h_ksize=3, channels=RB_CHANNELS, padding='REFLECT', strides=(2,2,1),
             name=f'{G}_Conv4'),
        ])

# output shape: batch_size, width, depth, transformations, height, channels
model.summary()

Model: "sequential_26"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ reshape_11 (Reshape)            │ (64, 64, 64, 1, 32, 4) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Lift_D4_Conv1 (RB3D_G_Conv)     │ (64, 33, 33, 8, 14, 4) │        10,136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ D4_Conv2 (RB3D_G_Conv)          │ (64, 17, 17, 8, 10, 4) │        57,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ D4_Conv3 (RB3D_G_Conv)          │ (64, 9, 9, 8, 6, 4)    │        34,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ D4_Conv4 (RB3D_G_Conv)          │ (64, 5, 5, 8, 2, 4)    │        11,528 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 113,888 (444.88 KB)

 Trainable params: 113,888 (444.88 KB)

 Non-trainable params: 0 (0.00 B)